In [ ]:
import matplotlib as mpl #画图用的库
import matplotlib.pyplot as plt
#下面这一句是为了可以在notebook中画图
%matplotlib inline
import numpy as np
import sklearn   #机器学习算法库
import pandas as pd #处理数据的库   
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras   #使用tensorflow中的keras
#import keras #单纯的使用keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, sklearn, pd, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist # 该数据集是黑白服装数据集
#拆分训练集和测试集
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
#将训练集拆分为训练集和验证集
#训练集共6万张图片，我们将前5000张作为验证集，后面所有的做训练集
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_train[0].dtype)
print(x_train[0]) # 是一个数据矩阵 28*28, 矩阵中的每一个数值都是uint8类型
print(y_train[0]) #这里的y值均为数字编码，非向量，所以后面定义模型损失函数为 sparse_categorical_crossentropy
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

In [ ]:
#在图像分类领域我们提升准确率的手段 归一化：
# 1.对训练数据进行归一化 2. 批归一化

# x = (x - u)/std  u为均值，std为方差
from sklearn.preprocessing import StandardScaler #使用sklearn中的StandardScaler实现训练数据归一化

scaler = StandardScaler()

#fit_transform:得到方差、均值、最大最小值然后数据进行归一化操作
#https://blog.csdn.net/youhuakongzhi/article/details/90519801
#x_train：先转为float32用于做除法，x_train本身为三维矩阵[None,28,28]，因为fit_transform要求二维数据所以需要转换为[None, 784]，再转回四维矩阵
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
#是因为在trainData的时候，已经使用fit()得到了整体的指标（均值，方差等）并被保存起来了后面验证集测试集可以使用，所以在测试集上直接transform()，使用之前的指标，
#如果在测试集上再进行fit()，由于两次的数据不一样，导致得到不同的指标，会使预测发生偏差，因为模型是针对之前的数据fit()出来
#的标准来训练的，而现在的数据是新的标准，会导致预测的不准确
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
#reshape(-1,1)表示（任意行，1列），这里个人认为设置里面什么参数影响不大，只要是转换为二维即可，反正最终要转换为三/四 维使用

In [ ]:
#tf.keras.models.Sequential()

model = keras.models.Sequential()


#使用深度卷积网络实现

model.add(keras.layers.Flatten(input_shape=[28,28]))
for _ in range(10):
    model.add(keras.layers.Dense(100,activation="selu"))# 激活函数selu自带数据归一化功能，在一定程度上也能缓解梯度消失问题

'''
#使用卷积神经网络实现
#激活函数这里使用了自带批归一化的selu函数来代替使用relu激活函数
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation="selu",input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation="selu"))
model.add(keras.layers.MaxPool2D(pool_size=2))

model.add(keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation="selu"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation="selu"))
model.add(keras.layers.MaxPool2D(pool_size=2))

model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation="selu"))
model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation="selu"))
model.add(keras.layers.MaxPool2D(pool_size=2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation="selu"))
'''

#softmax层输出
model.add(keras.layers.Dense(10,activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam", #optimizer="sgd", 优化算法一般来说我们无脑用adam即可
             metrics=["accuracy"])

#查看上面建立的模型架构信息
model.summary()

In [ ]:
#fit用于训练
history=model.fit(x_train_scaled, y_train, epochs=10, #epochs用于遍历训练集次数
                  validation_data=(x_valid_scaled,y_valid),#加入验证集，每隔一段时间就对验证集进行验证
                 )

In [ ]:
#将上面history中的数据指标用一张图来表示
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5)) #设置图的大小
    plt.grid(True) #显示网格
    plt.gca().set_ylim(0,1) #设置y轴范围
    plt.show()
plot_learning_curves(history)


In [ ]:
#测试集上进行测试评估一下
model.evaluate(x_test_scaled,y_test)

In [ ]:
tf.saved_model.save(model,'./keras_saved_graph')#保存 savedModel



In [ ]:
!saved_model_cli show --dir ./keras_saved_graph --all

In [ ]:
!saved_model_cli show --dir ./keras_saved_graph --tag_set serve --signature_def serving_default

In [ ]:
!saved_model_cli run --dir ./keras_saved_graph --tag_set serve --signature_def serving_default --input_exprs 'flatten_1_input=np.ones((2,28,28))'

In [ ]:
loaded_saved_model = tf.saved_model.load('./keras_saved_graph/')#加载本地的模型
print(list(loaded_saved_model.signatures.keys()))#打印所有的模型签名

In [ ]:
#通过上面的签名获取一个函数句柄inference
inference = loaded_saved_model.signatures['serving_default']
print(inference)
print(inference.structured_outputs)#输出dense_21,与前面的saved_model_cli命令输出一致

In [ ]:
#使用inference对象来做一个推理
resultes = inference(tf.constant(x_test_scaled[0:1]))#向这个句柄中传入test集合中的第一个图像输入
print(resultes)#输出为一个字典
print(resultes['dense_21'])
